In [ ]:
import base64
import pickle
import os
import pandas as pd
from io import BytesIO
import requests
from ipywidgets import widgets
from IPython.display import HTML, SVG, Image, Javascript, display
from threading import Thread
from datetime import datetime
import time

In [ ]:
pd.__version__

In [ ]:
from rdkit import Chem, RDConfig
from rdkit.Chem.Draw import rdMolDraw2D, IPythonConsole, MolsToGridImage

In [ ]:
os.getcwd()

In [ ]:
with open("JupyterDrawOptions.json") as hnd:
    rdMolDraw2D.UpdateMolDrawOptionsFromJSON(IPythonConsole.drawOptions, hnd.read())

In [ ]:
IPythonConsole.ipython_showProperties = True
IPythonConsole.ipython_useSVG = False

In [ ]:
from rdkit.Chem import PandasTools

In [ ]:
bilastine_pubchem_url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/bilastine/SDF"

In [ ]:
response = requests.get(bilastine_pubchem_url, allow_redirects=True)

In [ ]:
assert response.status_code == 200

In [ ]:
bilastine_sdf = response.content

In [ ]:
with BytesIO(bilastine_sdf) as buf:
    with Chem.ForwardSDMolSupplier(buf) as suppl:
        bilastine = next(suppl)

In [ ]:
bilastine

In [ ]:
with BytesIO(bilastine_sdf) as buf:
    df = PandasTools.LoadSDF(buf)

In [ ]:
romol = df.ROMol
df.drop(columns=["ROMol"], inplace=True)
df.insert(0, "ROMol", romol)

The size of the PNG molecule now honors `PandasTools.molSize`:

In [ ]:
df

In [ ]:
IPythonConsole.InteractiveRenderer.setEnabled()

In [ ]:
PandasTools.molRepresentation = "svg"

The same is true for SVG:

In [ ]:
df

`PandasTools.LoadSDF()` automatically calls `RenderImagesInAllDataFrames`, so let's revert that to see what happens when the global flag is not set.

As expected, now the molecule is not shown.

In [ ]:
PandasTools.RenderImagesInAllDataFrames(False)

In [ ]:
df2 = pd.DataFrame({"mol": [bilastine]})

In [ ]:
df2

The DataFrame should be displayed with molecules after calling `ChangeMoleculeRendering` on it, and indeed it is:

In [ ]:
PandasTools.ChangeMoleculeRendering(df2)

In [ ]:
df2

In [ ]:
df3 = pd.DataFrame({"mol": [bilastine]})

In [ ]:
df3

Also the global flag works...

In [ ]:
PandasTools.RenderImagesInAllDataFrames()

In [ ]:
df3

...and it can be reverted:

In [ ]:
PandasTools.RenderImagesInAllDataFrames(False)

In [ ]:
df3

In [ ]:
PandasTools.RenderImagesInAllDataFrames()

In [ ]:
def counter_box():
    return widgets.HTML(value=f"<code>0</code>").add_class("Chem_Mol__str__counter")

In [ ]:
def reposition_counter_box(widget):
    display(Javascript("""
const div = document.querySelector('div[class*=Chem_Mol__str__counter]');
const label = div.querySelector('label');
const titleDiv = document.createElement('div');
titleDiv.style = 'margin-bottom: 4px';
titleDiv.innerHTML = '<code>Chem.Mol.__str.__</code> call counter';
const countDiv = div.querySelector('div[class=widget-html-content]');
countDiv.style.textAlign = 'right';
div.insertBefore(titleDiv, label);
const TOP_PADDING = 120;
const RIGHT_PADDING = 100;
const viewPortWidth = window.innerWidth || document.body.clientWidth;
div.style = `display: block; position: absolute; ` +
            `border: 2px solid black; background-color: white; ` +
            `padding: 8px; top: ${TOP_PADDING}px; z-index: 9999;`;
document.body.appendChild(div);
const rect = div.getBoundingClientRect();
const left = Math.round(viewPortWidth - rect.width - RIGHT_PADDING);
div.style.left = `${left}px`;
"""))

In [ ]:
counter_widget = counter_box()

In [ ]:
counter_widget

In [ ]:
reposition_counter_box(counter_widget)

We patch `Chem.Mol.__str__` to show that it gets called for all variables defined in the Jupyter Notebook after each cell is executed:

In [ ]:
def new_str_method(x):
    """Increment call count, then call the original method"""
    new_str_method.__call_count += 1
    return orig_str_method(x)

In [ ]:
# patch Chem.Mol.__str__
# (which had been previously patched by PandasTools)
if not hasattr(Chem.Mol.__str__, "__call_count"):
    orig_str_method = Chem.Mol.__str__
    Chem.Mol.__str__ = new_str_method
    setattr(new_str_method, "__call_count", 0)

In [ ]:
def update_counter_box(counter_widget, new_str_method):
    UPDATE_INTERVAL_SECONDS = 1
    while not hasattr(update_counter_box, "stop"):
        counter_widget.value = f"<code>{new_str_method.__call_count}</code>"
        time.sleep(UPDATE_INTERVAL_SECONDS)

We start a background thread that updates the counter box every second:

In [ ]:
t = Thread(target=update_counter_box, args=(counter_widget, new_str_method), daemon=True)

In [ ]:
t.start()

We load 5000 molecules:

In [ ]:
smi5000 = os.path.join(RDConfig.RDDataDir, "NCI", "first_5K.smi")

In [ ]:
assert os.path.exists(smi5000)

In [ ]:
with Chem.SmilesMolSupplier(smi5000, delimiter="\t") as suppl:
    mols = [mol for mol in suppl if mol]

In [ ]:
len(mols)

Now for each of the 4991 molecules we define a variable in the Notebook's namespace:

In [ ]:
for i in range(len(mols)):
    exec(f"mol{i} = mols[{i}]")

Looking at the counter box, you may see that the counter was incremented for each molecule added to the Notebook as a separate variable.

However, now the counter updates are instantaneous, as `Chem.Mol.__str__` only prints a string as per its default behavior, which takes negligible time.

In [ ]:
datetime.now()

In [ ]:
datetime.now()

In [ ]:
df4991 = pd.DataFrame({"mols": mols})

In [ ]:
df4991

## Enjoy your snappy `PandasTools`-enabled Jupter Notebook!

In [ ]:
setattr(update_counter_box, "stop", True)
counter_widget.close()